https://towardsdatascience.com/extract-knowledge-from-text-end-to-end-information-extraction-pipeline-with-spacy-and-neo4j-502b2b1e0754

https://github.com/tomasonjo/blogs/blob/master/ie_pipeline/SpaCy_informationextraction.ipynb

In [ ]:
!pip install crosslingual-coreference > /dev/null

In [10]:
!pip install spacy-transformers > /dev/null

In [13]:
!pip install spacy > /dev/null

In [1]:
!pip install transformers > /dev/null

In [2]:
import transformers

transformers.__version__

'4.20.0'

In [1]:
!pip install spacy-transformers wikipedia neo4j > /dev/null
!pip install --upgrade google-cloud-storage > /dev/null
!python -m spacy download en_core_web_sm > /dev/null

ERROR: Ignored the following versions that require a different python version: 0.2.0 Requires-Python ==3.6; 0.2.6 Requires-Python >=3.7.1,<3.8.0
ERROR: Could not find a version that satisfies the requirement torch<1.11.0,>=1.10.0 (from crosslingual-coreference) (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch<1.11.0,>=1.10.0
2023-06-21 04:30:57.616001: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 04:30:58.765261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import spacy
import crosslingual_coreference

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# Add rebel component https://github.com/Babelscape/rebel/blob/main/spacy_component.py
import requests
import re
import hashlib
from spacy import Language
from typing import List

from spacy.tokens import Doc, Span

from transformers import pipeline

In [5]:
def call_wiki_api(item):
  try:
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
    data = requests.get(url).json()
    # Return the first id (Could upgrade this in the future)
    return data['search'][0]['id']
  except:
    return 'id-less'


In [6]:
def extract_triplets(text):
    """
    Function to parse the generated text and extract the triplets
    """
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("", "").replace("", "").replace("", "").split():
        if token == "":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})

    return triplets

In [7]:
@Language.factory(
    "rebel",
    requires=["doc.sents"],
    assigns=["doc._.rel"],
    default_config={
        "model_name": "Babelscape/rebel-large",
        "device": 0,
    },
)
class RebelComponent:
    def __init__(
        self,
        nlp,
        name,
        model_name: str,
        device: int,
    ):
        assert model_name is not None, ""
        self.triplet_extractor = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)
        self.entity_mapping = {}
        # Register custom extension on the Doc
        if not Doc.has_extension("rel"):
          Doc.set_extension("rel", default={})

    def get_wiki_id(self, item: str):
        mapping = self.entity_mapping.get(item)
        if mapping:
          return mapping
        else:
          res = call_wiki_api(item)
          self.entity_mapping[item] = res
          return res


    def _generate_triplets(self, sent: Span) -> List[dict]:
          output_ids = self.triplet_extractor(sent.text, return_tensors=True, return_text=False)[0]["generated_token_ids"]["output_ids"]
          extracted_text = self.triplet_extractor.tokenizer.batch_decode(output_ids[0])
          extracted_triplets = extract_triplets(extracted_text[0])
          return extracted_triplets

    def set_annotations(self, doc: Doc, triplets: List[dict]):
        for triplet in triplets:

            # Remove self-loops (relationships that start and end at the entity)
            if triplet['head'] == triplet['tail']:
                continue

            # Use regex to search for entities
            head_span = re.search(triplet["head"], doc.text)
            tail_span = re.search(triplet["tail"], doc.text)

            # Skip the relation if both head and tail entities are not present in the text
            # Sometimes the Rebel model hallucinates some entities
            if not head_span or not tail_span:
              continue

            index = hashlib.sha1("".join([triplet['head'], triplet['tail'], triplet['type']]).encode('utf-8')).hexdigest()
            if index not in doc._.rel:
                # Get wiki ids and store results
                doc._.rel[index] = {"relation": triplet["type"], "head_span": {'text': triplet['head'], 'id': self.get_wiki_id(triplet['head'])}, "tail_span": {'text': triplet['tail'], 'id': self.get_wiki_id(triplet['tail'])}}

    def __call__(self, doc: Doc) -> Doc:
        for sent in doc.sents:
            sentence_triplets = self._generate_triplets(sent)
            self.set_annotations(doc, sentence_triplets)
        return doc

In [8]:
DEVICE = -1 # Number of the GPU, -1 if want to use CPU

In [9]:
# Add coreference resolution model
coref = spacy.load('en_core_web_sm')

coref.add_pipe("xx_coref",
               config={"chunk_size": 2500,
                       "chunk_overlap": 2,
                       "device": DEVICE})

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of the model checkpoint at nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large and are newly initiali

In [10]:
coref("What is the subject of this line")

What is the subject of this line

In [11]:
# Define rel extraction model
rel_ext = spacy.load('en_core_web_sm')

rel_ext.add_pipe("rebel", config={
    'device':DEVICE, # Number of the GPU, -1 if want to use CPU
    'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
    )

In [13]:
input_text = "Why no data is getting populated."

coref_text = coref(input_text)._.resolved_text

'Why no data is getting populated.'

In [14]:
doc = rel_ext(coref_text)

IndexError: ignored

In [29]:
doc._.rel

{}

In [10]:
for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")

In [16]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large',
                             tokenizer='Babelscape/rebel-large')
text = "Punta Cana is a resort town in the municipality of Higuey, in La Altagracia Province, the eastern most province of the Dominican Republic"

In [17]:
extracted_data = triplet_extractor(text,
                  return_tensors=True,
                  return_text=False)[0]

In [32]:
extracted_data

{'generated_token_ids': {'output_ids': tensor([[[    0, 50267,   221, 20339,  2615,   102,  1437, 50266,  1587,  7330,
             1073, 13249,   493, 16517,  1437, 50265,  2034,    11,     5,  6833,
            15752, 10014,  1437, 50266, 18978,  3497,  1437, 50265,   247,  1437,
            50267, 19664,  1780,   219,  1437, 50266,  1587,  7330,  1073, 13249,
              493, 16517,  1437, 50265,  2034,    11,     5,  6833, 15752, 10014,
             1437, 50266, 18978,  3497,  1437, 50265,   247,  1437, 50267,  1587,
             7330,  1073, 13249,   493, 16517,  1437, 50266, 18978,  3497,  1437,
            50265,   247,  1437, 50267, 18978,  3497,  1437, 50266,  1587,  7330,
             1073, 13249,   493, 16517,  1437, 50265,  6308,  6833, 15752, 10014,
                2]]])}}

In [37]:
triplet_extractor.tokenizer.decode(extracted_data)

TypeError: ignored

In [ ]:
import pandas as pd
import wikipedia
from neo4j import GraphDatabase

In [ ]:

# Define Neo4j connection
host = 'bolt://3.236.134.179:7687'
user = 'neo4j'
password = 'writer-calibers-steels'
driver = GraphDatabase.driver(host,auth=(user, password))

import_query = """
UNWIND $data AS row
MERGE (h:Entity {id: CASE WHEN NOT row.head_span.id = 'id-less' THEN row.head_span.id ELSE row.head_span.text END})
ON CREATE SET h.text = row.head_span.text
MERGE (t:Entity {id: CASE WHEN NOT row.tail_span.id = 'id-less' THEN row.tail_span.id ELSE row.tail_span.text END})
ON CREATE SET t.text = row.tail_span.text
WITH row, h, t
CALL apoc.merge.relationship(h, toUpper(replace(row.relation,' ', '_')),
  {},
  {},
  t,
  {}
)
YIELD rel
RETURN distinct 'done' AS result;
"""

In [18]:
def store_wikipedia_summary(page):
  try:
    input_text = wikipedia.page(page).summary
    coref_text = coref(input_text)._.resolved_text
    doc = rel_ext(coref_text)
    params = [rel_dict for value, rel_dict in doc._.rel.items()]
    run_query(import_query, {'data': params})
  except Exception as e:
    print(f"Couldn't parse text for {page} due to {e}")


In [19]:
ladies = ["Jennifer Doudna", "Rachel Carson", "Sara Seager OC", "Gertrude Elion", "Rita Levi-Montalcini"]

for l in ladies:
  print(f"Parsing {l}")
  store_wikipedia_summary(l)

Parsing Jennifer Doudna
Couldn't parse text for Jennifer Doudna due to name 'wikipedia' is not defined
Parsing Rachel Carson
Couldn't parse text for Rachel Carson due to name 'wikipedia' is not defined
Parsing Sara Seager OC
Couldn't parse text for Sara Seager OC due to name 'wikipedia' is not defined
Parsing Gertrude Elion
Couldn't parse text for Gertrude Elion due to name 'wikipedia' is not defined
Parsing Rita Levi-Montalcini
Couldn't parse text for Rita Levi-Montalcini due to name 'wikipedia' is not defined


In [ ]:
run_query("""
CALL apoc.periodic.iterate("
  MATCH (e:Entity)
  WHERE e.id STARTS WITH 'Q'
  RETURN e
","
  // Prepare a SparQL query
  WITH 'SELECT * WHERE{ ?item rdfs:label ?name . filter (?item = wd:' + e.id + ') filter (lang(?name) = \\"en\\") ' +
     'OPTIONAL {?item wdt:P31 [rdfs:label ?label] .filter(lang(?label)=\\"en\\")}}' AS sparql, e
  // make a request to Wikidata
  CALL apoc.load.jsonParams(
    'https://query.wikidata.org/sparql?query=' +
      sparql,
      { Accept: 'application/sparql-results+json'}, null)
  YIELD value
  UNWIND value['results']['bindings'] as row
  SET e.wikipedia_name = row.name.value
  WITH e, row.label.value AS label
  MERGE (c:Class {id:label})
  MERGE (e)-[:INSTANCE_OF]->(c)
  RETURN distinct 'done'", {batchSize:1, retry:1})
""")